In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

/home/aditywardhanm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
#Constants
savename = ""
num_classes = 38
learning_rate = 3e-4
batch_size = 128
num_epochs = 20

In [ ]:
"""
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
source_folder = "/" #Path to the folder with the images
train_folder = "/" #Path to the folder where you want to store the train images
test_folder = "/" #Path to the folder where you want to store the test images

# Create train and test folders if they don't exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get a list of all subfolders in the source folder
subfolders = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]

for subfolder in subfolders:
    # Create subfolders in train and test folders
    os.makedirs(os.path.join(train_folder, subfolder), exist_ok=True)
    os.makedirs(os.path.join(test_folder, subfolder), exist_ok=True)

    # Get a list of all image files in the subfolder
    image_files = [f for f in os.listdir(os.path.join(source_folder, subfolder)) if os.path.isfile(os.path.join(source_folder, subfolder, f))]

    # Split the dataset into train and test sets
    train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42) # SPLIT SIZE CHANGE HERE

    # Move files to respective folders
    for file_name in train_files:
        source_path = os.path.join(source_folder, subfolder, file_name)
        destination_path = os.path.join(train_folder, subfolder, file_name)
        shutil.copy(source_path, destination_path)

    for file_name in test_files:
        source_path = os.path.join(source_folder, subfolder, file_name)
        destination_path = os.path.join(test_folder, subfolder, file_name)
        shutil.copy(source_path, destination_path)
"""

In [3]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define your data directories====Non-Augmented
train_dir = "/home/user/Training"  # Location of the Training Data
val_dir = "/home/user/Validation"  # Location of the Validation Data

# Define mean and standard deviation for ImageNet
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Add normalization to your transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Create datasets
train_dataset = ImageFolder(root=train_dir, transform=train_transform)
val_dataset = ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Check the number of images in the training dataloader
num_train_images = len(train_loader.dataset)
print(f"Number of images in the training dataloader: {num_train_images}")

# Check the number of images in the validation dataloader
num_val_images = len(val_loader.dataset)
print(f"Number of images in the validation dataloader: {num_val_images}")

Number of images in the training dataloader: 37998
Number of images in the validation dataloader: 16307


In [12]:
import torch
import torch.nn as nn
from transformers import AutoModelForImageClassification, AutoConfig

class CustomPM(nn.Module):
    def __init__(self, num_classes, pretrained_model="facebook/convnextv2-base-1k-224", hidden_size=256):
        super(CustomPM, self).__init__()

        config = AutoConfig.from_pretrained(pretrained_model)
        self.cvnt = AutoModelForImageClassification.from_pretrained(pretrained_model, config=config)
        
        # Freeze ConvNeXt-V2 model
        for param in self.cvnt.parameters():
            param.requires_grad = False

        # Extracting the original classifier layer
        original_classifier = self.cvnt.classifier

        self.cvnt.classifier = nn.Sequential(
            nn.Linear(original_classifier.in_features, hidden_size),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )        
    def forward(self, images):
        outputs = self.cvnt(images)
        logits = outputs.logits
        return logits

model = CustomPM(num_classes)
model.to(device)

CustomPM(
  (pm): ConvNextV2ForImageClassification(
    (convnextv2): ConvNextV2Model(
      (embeddings): ConvNextV2Embeddings(
        (patch_embeddings): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (layernorm): ConvNextV2LayerNorm()
      )
      (encoder): ConvNextV2Encoder(
        (stages): ModuleList(
          (0): ConvNextV2Stage(
            (downsampling_layer): Identity()
            (layers): Sequential(
              (0): ConvNextV2Layer(
                (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
                (layernorm): ConvNextV2LayerNorm()
                (pwconv1): Linear(in_features=128, out_features=512, bias=True)
                (act): GELUActivation()
                (grn): ConvNextV2GRN()
                (pwconv2): Linear(in_features=512, out_features=128, bias=True)
                (drop_path): Identity()
              )
              (1): ConvNextV2Layer(
                (dwconv): Conv2d(128, 128

In [13]:
import os
from torch.optim import AdamW
from lion_pytorch import Lion
from sklearn.metrics import precision_score, recall_score, f1_score

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

#Comment out the optimizer you don't want to use

optimizer = Lion(model.parameters(), lr=learning_rate)
#optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()  # Set model to training mode
    train_loss = 0.0
    correct_train = 0

    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
    
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = correct_train / len(train_loader.dataset) * 100
    
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()

        # Collect predictions and labels for later evaluation
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct_val / len(val_loader.dataset) * 100

    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

    # Print the epoch statistics
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Save the model
final_checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
torch.save(final_checkpoint, savename + '.pt')

In [4]:
pip install pyinstaller

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.0/663.0 KB 220.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 KB 288.7 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pyinstaller --onefile /home/aditywardhanm/script.py

NameError: name 'pyinstaller' is not defined

In [1]:
import torch
import torch.nn as nn
import torchvision
import tkinter
from torchvision.transforms import functional as F
import torchvision.transforms as transforms
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from transformers import AutoModelForImageClassification, AutoConfig

app = tkinter.Tk()
app.geometry("250x325")
app.title("Plant Disease Detector")
class_mapping = {0: 'Apple___Apple_scab', 1: 'Apple___Black_rot', 2: 'Apple___Cedar_apple_rust', 3: 'Apple___healthy', 4: 'Blueberry___healthy', 5: 'Cherry___Powdery_mildew', 6: 'Cherry___healthy', 7: 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 8: 'Corn___Common_rust', 9: 'Corn___Northern_Leaf_Blight', 10: 'Corn___healthy', 11: 'Grape___Black_rot', 12: 'Grape___Esca_(Black_Measles)', 13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 14: 'Grape___healthy', 15: 'Orange___Haunglongbing_(Citrus_greening)', 16: 'Peach___Bacterial_spot', 17: 'Peach___healthy', 18: 'Pepper,_bell___Bacterial_spot', 19: 'Pepper,_bell___healthy', 20: 'Potato___Early_blight', 21: 'Potato___Late_blight', 22: 'Potato___healthy', 23: 'Raspberry___healthy', 24: 'Soybean___healthy', 25: 'Squash___Powdery_mildew', 26: 'Strawberry___Leaf_scorch', 27: 'Strawberry___healthy', 28: 'Tomato___Bacterial_spot', 29: 'Tomato___Early_blight', 30: 'Tomato___Late_blight', 31: 'Tomato___Leaf_Mold', 32: 'Tomato___Septoria_leaf_spot', 33: 'Tomato___Spider_mites Two-spotted_spider_mite', 34: 'Tomato___Target_Spot', 35: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 36: 'Tomato___Tomato_mosaic_virus', 37: 'Tomato___healthy'}
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 38

def getImage():
    path = filedialog.askopenfilename()
    return path

def preprocess(path):
    # Mean and standard deviation for ImageNet
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    t_image= transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(),transforms.Normalize(mean, std)]) 
    image = Image.open(path).convert("RGB")
    image = t_image(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    return image

class CustomPM(nn.Module):
    def __init__(self, num_classes, pretrained_model="facebook/convnextv2-base-1k-224", hidden_size=256):
        super(CustomPM, self).__init__()

        config = AutoConfig.from_pretrained(pretrained_model)
        self.cvnt = AutoModelForImageClassification.from_pretrained(pretrained_model, config=config)        
        # Freeze ConvNeXt-V2 model
        for param in self.cvnt.parameters():
            param.requires_grad = False

        # Extracting the original classifier layer
        original_classifier = self.cvnt.classifier

        self.cvnt.classifier = nn.Sequential(
            nn.Linear(original_classifier.in_features, hidden_size),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )        
    def forward(self, images):
        outputs = self.cvnt(images)
        logits = outputs.logits
        return logits
    
def classify(image):
    model = CustomPM(num_classes)
    model.to(device)

    # Load the saved model
    checkpoint = torch.load('/home/aditywardhanm/ConvNeXtV2_Lion_Aug.pt')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    # Use the model to predict the class
    with torch.no_grad():
        output = model(image)

    # Get the predicted class
    _, predicted_class = torch.max(output, 1)

    # Map the predicted class index to the class name
    predicted_class_name = class_mapping.get(predicted_class.item(), 'Unknown Class')

    return predicted_class_name

def image_classfier():
    path = getImage()
    input_image= preprocess(path)
    results= classify(input_image)

upload_button = tkinter.Button(text="Upload", command= image_classfier)
upload_button.place(relx=0.5, rely=0.5, anchor='center')

result_label = tkinter.Label(text="")
result_label.place(relx=0.5, rely=0.6, anchor="center")

app.mainloop

/home/aditywardhanm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/aditywardhanm/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


<bound method Misc.mainloop of <tkinter.Tk object .>>